# Test trajectory planning in generated map

In [1]:
%matplotlib widget
import numpy as np
import os
import time
import plotly.graph_objects as go
from copy import deepcopy

import planeslam.io as io
from planeslam.scan import pc_to_scan
from planeslam.general import NED_to_ENU, trajectory_plot_trace
from planeslam.geometry.util import quat_to_R

os.environ['KMP_DUPLICATE_LIB_OK']='True'

%load_ext autoreload
%autoreload 2
%load_ext line_profiler

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
np.set_printoptions(suppress=True)

### Load AirSim data

In [3]:
# Read in point cloud data
binpath = os.path.join(os.getcwd(), '..', 'data', 'airsim', 'blocks_sub_2_1200_samples_5hz_noyaw', 'lidar', 'Drone0')
PC_data = io.read_lidar_bin(binpath)

# Read in ground-truth poses (in drone local frame)
posepath = os.path.join(os.getcwd(), '..', 'data', 'airsim', 'blocks_sub_2_1200_samples_5hz_noyaw', 'poses', 'Drone0')
drone_positions, drone_orientations = io.read_poses(posepath)

In [4]:
# Subsample data
sub_factor = 5
PC_data = PC_data[::sub_factor]
drone_positions = drone_positions[::sub_factor]
drone_orientations = drone_orientations[::sub_factor]

In [5]:
# Plot ground-truth trajectory
gt_traj_trace = go.Scatter3d(x=drone_positions[:,0], y=drone_positions[:,1], z=drone_positions[:,2], 
    marker=dict(size=2, color='orange'), hovertext=np.arange(len(drone_positions)))
fig = go.Figure(data=gt_traj_trace)
fig.update_layout(width=1000, height=600, scene=dict(aspectmode='data'))
fig.show()

In [6]:
# Convert to ENU
num_scans = len(PC_data)

for i in range(num_scans):
    PC_data[i] = NED_to_ENU(PC_data[i])

drone_positions = NED_to_ENU(drone_positions)
drone_orientations = NED_to_ENU(drone_orientations)

drone_rotations = np.zeros((3,3,num_scans))
for i in range(num_scans):
    drone_rotations[:,:,i] = quat_to_R(drone_orientations[i])

In [7]:
# Extract scans
num_scans = len(PC_data)
scans = []
avg_extraction_time = 0
for i in range(num_scans):
    start_time = time.time()
    scans.append(pc_to_scan(PC_data[i]))
    avg_extraction_time += time.time() - start_time
avg_extraction_time /= num_scans
print(avg_extraction_time)

0.03490544060866038


Generate Map

In [8]:
# Initialize transformed scans
scans_transformed = []
for i in range(num_scans):
    scans_transformed.append(deepcopy(scans[i]))
    scans_transformed[i].transform(drone_rotations[:,:,i], drone_positions[i])

In [9]:
merged = scans_transformed[0]

for s in scans_transformed[1:76] + scans_transformed[77:-5]:
    merged = merged.merge(s, dist_thresh=7.5)
    merged.reduce_inside(p2p_dist_thresh=5)
    merged.remove_small_planes(area_thresh=5.0)
    merged.fuse_edges(vertex_merge_thresh=2.0)

In [17]:
from scipy.spatial.transform import Rotation as R

r = R.from_euler('y', -0.4, degrees=True)
merged.transform(r.as_matrix(), np.zeros(3))

In [18]:
# Plot merge
fig = go.Figure(data=merged.plot_trace(colors=['blue']))
fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data'))
fig.show()

High-level planner (RRT)

In [11]:
root_pos = np.array([40.0, -75.0, 2.0])

G_pos = root_pos
G_edges = []

RRT_radius = 5
world_bounds = np.array([[-26, 204],  # x_min, x_max
                         [-122, 19],  # y_min, y_max,
                         [-1, 24]])   # z_min, z_max

In [12]:
N_iters = 1000
vertex_ct = 1

start_time = time.time()

for i in range(N_iters):
    # Sample a random position in bounds
    rand_sample = np.random.random_sample(size=3)
    rand_pos = world_bounds[:,0] + rand_sample * np.diff(world_bounds).flatten()

    # Find nearest vertex in G
    if G_pos.ndim > 1:
        dists = np.linalg.norm(G_pos - rand_pos, axis=1)
        nearest = np.argmin(dists)
        nearest_pos = G_pos[nearest]
        dist_nearest = dists[nearest]
    else:
        nearest = 0
        nearest_pos = G_pos
        dist_nearest = np.linalg.norm(G_pos - rand_pos)

    # Find new vertex along path and check for collision
    new_pos = nearest_pos + (RRT_radius / dist_nearest) * (rand_pos - nearest_pos)
    line = np.vstack((nearest_pos, new_pos))
    collision = False
    start_time = time.time()
    for plane in merged.planes:
        if plane.check_line_intersect(line):
            collision = True
    
    # If no collision, add new vertex to G
    if not collision:
        G_pos = np.vstack((G_pos, new_pos))
        G_edges.append((nearest, vertex_ct))
        vertex_ct += 1
        
print("elapsed: ", time.time() - start_time)

elapsed:  0.0010004043579101562


In [38]:
# Visualize
edge_x = []
edge_y = []
edge_z = []
for edge in G_edges:
    x0, y0, z0 = G_pos[edge[0]]
    x1, y1, z1 = G_pos[edge[1]]
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)
    edge_z.append(z0)
    edge_z.append(z1)
    edge_z.append(None)

edge_trace = go.Scatter3d(x=edge_x, y=edge_y, z=edge_z, mode='lines', line=dict(width=10, color='orange'))
node_trace = go.Scatter3d(x=G_pos[:,0], y=G_pos[:,1], z=G_pos[:,2], mode='markers', marker=dict(size=10, color='coral'))

In [39]:
camera = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=0, y=-2.0, z=1.0)
)

fig = go.Figure(data=merged.plot_trace(colors=['blue'])+[edge_trace, node_trace])
fig.update_layout(width=4000, height=2500, scene=dict(aspectmode='data', xaxis=dict(visible=False), yaxis=dict(visible=False), zaxis=dict(visible=False)),
    scene_camera=camera)
fig.show()

In [40]:
fig.write_image("images/rrt.png", width=4000, height=2500)

Trajectory planning

In [ ]:
from planeslam.geometry.box3d import Box3D
from planeslam.planning.LPM import LPM
from planeslam.planning.reach_planner import ReachPlanner
import planeslam.planning.params as params
from planeslam.planning.zonotope import Zonotope

In [ ]:
from planeslam.planning.reachability import compute_FRS, generate_collision_constraints_FRS, check_collision_constraints, check_collision_constraints_vectorized

In [ ]:
LPM_file = os.path.join(os.getcwd(),'..', 'data', 'planning_models', 'quadrotor_linear_planning_model.mat')

p_0 = np.array([40.0, -75.0, 2.0])[:,None]
#r_body = Box3D(np.zeros(3), np.eye(3)/2)
r_body = Zonotope(np.zeros((3,1)), 2*np.eye(3))

planner = ReachPlanner(LPM_file, p_0, map=merged, r_body=r_body)

In [ ]:
waypts = np.array([[40, -75, 2.0],
                   [60, -45, 2.0],
                   [80, -65, 2.0],
                   [100, -65, 2.0],
                   [120, -80, 2.0],
                   [140, -45, 2.0]])
waypt_idx = 1

In [ ]:
# Plot merge
fig = go.Figure(data=merged.plot_trace(colors=['blue'])+[go.Scatter3d(x=waypts[:,0], y=waypts[:,1], z=waypts[:,2], marker=dict(size=5, color='orange'))])
fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
# "Controller" loop frequency 
# Determined by trajectory discretization
# Currently the controller just pops off setpoints from the planner trajectories to save for plotting
controller_hz = 10  # 0.1 s

# Planner loop frequency
# Determines time between replans
planner_hz = 2  # 0.5 s

t_sim = 0  # counter for simulation time
trajectory = None
traj_idx = 0
done = False

# Initialize state
p = p_0
v = np.zeros((3,1))
a = np.zeros((3,1))
# State history
P = []
planning_times = []

# Set initial goal
planner.p_goal = waypts[waypt_idx,:][:,None]

while not done:

    # Planner
    if t_sim % int(controller_hz/planner_hz) == 0:
        # Replan
        print("Replanning: t_sim = ", t_sim)
        start_t = time.time()
        trajectory = planner.replan((p,v,a))
        planning_times.append(time.time() - start_t)
        traj_idx = 0

        if trajectory is None:
            print("Replanning failed, stopping")
            break

    # Controller
    # Pop off next setpoint
    p = trajectory[0][:,traj_idx][:,None]
    v = trajectory[1][:,traj_idx][:,None]
    a = trajectory[2][:,traj_idx][:,None]
    P.append(p)
    traj_idx += 1

    if np.linalg.norm(p - planner.p_goal) < params.R_GOAL_REACHED:
        print("Waypoint reached")
        waypt_idx += 1
        if waypt_idx < len(waypts):
            planner.p_goal = waypts[waypt_idx,:][:,None]
        else:
            print("Done.")
            done = True

    t_sim += 1

P = np.hstack(P)
print("Average planning time: ", np.mean(planning_times), " seconds")

In [ ]:
np.std(planning_times)

In [ ]:
# Plot trajectory
fig = go.Figure(data=merged.plot_trace(colors=['blue'])
    +[go.Scatter3d(x=waypts[:,0], y=waypts[:,1], z=waypts[:,2], marker=dict(size=5, color='orange'))]
    +[go.Scatter3d(x=P[0,:], y=P[1,:], z=P[2,:], marker=dict(size=2, color='red'))])
fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data', xaxis=dict(visible=False), yaxis=dict(visible=False), zaxis=dict(visible=False)))
fig.show()

In [ ]:
# Compute FRS from initial conditions
FRS = compute_FRS(planner.LPM, planner.p_0, planner.v_0, planner.a_0)

# Generate collision constraints
# NOTE: For now, only generate constraints for final element of FRS
nearby_obs = [planner.zono_map[i] for i in planner.get_nearby_obs_idxs()]
A_con, b_con = generate_collision_constraints_FRS(FRS, nearby_obs)

In [ ]:
lpm = LPM(LPM_file)
v_0 = np.zeros((3,1))
a_0 = np.zeros((3,1))
v_peak = np.array([3, 0, 0])[:,None]
k = np.hstack((v_0, a_0, v_peak))
#(p_0 + lpm.compute_positions(k)).T

In [ ]:
check_collision_constraints(A_con, b_con, v_peak)

In [ ]:
check_collision_constraints_vectorized(A_con, b_con, v_peak)

In [ ]:
%lprun -f check_collision_constraints check_collision_constraints(A_con, b_con, v_peak)

In [ ]:
%lprun -f check_collision_constraints_vectorized check_collision_constraints_vectorized(A_con, b_con, v_peak)

In [ ]:
import matplotlib.pyplot as plt
from planeslam.planning.reachability import compute_PRS, compute_FRS, generate_collision_constraints, check_collision_constraints

lpm = LPM(LPM_file)

In [ ]:
v_0 = np.zeros((3,1))
a_0 = np.zeros((3,1))
FRS = compute_FRS(lpm, p_0, v_0, a_0)

In [ ]:
fig, ax = plt.subplots()

for zono in FRS[1:]:
    #print(zono)
    zono_2D = zono.index([0,1])
    zono_2D.plot(ax=ax, alpha=0.05)

In [ ]:
zono_map = []
for plane in merged.planes: 
    c = plane.center[:,None]
    G = np.diff(plane.vertices[:3], axis=0).T / 2
    zono_map.append(Zonotope(c, G))

In [ ]:
zono_map[0].index([0,1]).plot()

In [ ]:
FRS[-1]

In [ ]:
c = FRS[-1].c[params.OBS_DIM]
G = FRS[-1].G

# Find columns of G which are nonzero in k_dim ("k-sliceable")
# - this forms a linear map from the parameter space to workspace
k_col = list(set(np.nonzero(G[params.K_DIM,:])[1]))
k_slc_G = G[params.OBS_DIM][:,k_col]

# "non-k-sliceable" generators - have constant contribution regardless of chosen trajectory parameter
k_no_slc_G = G[params.OBS_DIM]
k_no_slc_G = np.delete(k_no_slc_G, k_col, axis=1)

In [ ]:
k_no_slc_G

In [ ]:
# Get current obstacle
obs = zono_map[0].Z

# Obstacle is "buffered" by non-k-sliceable part of FRS
buff_obs_c = obs[:,0][:,None] - c
buff_obs_G = np.hstack((obs[:,1:], k_no_slc_G))
#buff_obs_G = remove_zero_columns(buff_obs_G)
buff_obs = Zonotope(buff_obs_c, buff_obs_G)

A_obs, b_obs = buff_obs.halfspace()

In [ ]:
A = A_obs @ k_slc_G
b = b_obs

In [ ]:
b

In [ ]:
A_con, b_con = generate_collision_constraints(FRS, [zono_map[0]])

In [ ]:
b_con[-1]

In [ ]:
v_peak = np.array([1, 1, -1.5])[:,None]
k = np.hstack((v_0, a_0, v_peak))
lpm.compute_positions(k)

In [ ]:
lambdas = v_peak / params.V_MAX
lambdas

In [ ]:
print(A_con[29], b_con[29])

In [ ]:
A_con[-1] @ lambdas - b_con[-1]

In [ ]:
check_collision_constraints(A_con, b_con, v_peak)

In [ ]:
P = np.hstack(P)

In [ ]:
# Plot trajectory
fig = go.Figure(data=merged.plot_trace()+[go.Scatter3d(x=P[0,:], y=P[1,:], z=P[2,:], marker=dict(size=2))])
fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
lpm = LPM(LPM_file)

k = np.array([[0, 0, 5],
              [0, 0, 5],
              [0, 0, 5]])
positions = lpm.compute_positions(k)

In [ ]:
v_0 = np.array([1,2,3])
a_0 = np.array([2,3,4])
k_0 = np.vstack((v_0, a_0))

In [ ]:
from planeslam.planning.reachability import compute_PRS

PRS = compute_PRS(lpm, v_0, a_0)

In [ ]:
z = Zonotope(np.zeros((3,1)), np.eye(3))
z.contains(np.array([0,0,1.1])[:,None])

Test generate collision constraints

In [ ]:
from planeslam.general import remove_zero_columns

In [ ]:
k_dim = np.array([6, 9])
obs_dim = np.array([0, 1])

In [ ]:
obs = np.array([[3, 1, 0], [-3, 0, 1]])

In [ ]:
c = np.array([5.43, 5.0])[:,None]
G = np.array([[0, 0, 0.0807, -0.0462, 0.0348, -0.0846], 
              [0, 0, 0.0462, 0.0807, -0.0846, -0.0348],
              [0, 0, -0.0757, -0.0433, 0.0371, -0.0902],
              [0, 0, -0.0433, -0.0757, -0.0902, -0.0371],
              [0, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0],
              [3, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0],
              [0, 3, 0, 0, 0, 0]])

In [ ]:
# Find columns of G which are nonzero in k_dim ("k-sliceable")
k_col = list(set(np.nonzero(G[k_dim,:])[1]))

In [ ]:
k_slc_G = G[obs_dim][:,k_col]

In [ ]:
# "non-k-sliceable" generators - have constant contribution regardless of chosen trajectory parameter
k_no_slc_G = G[obs_dim]
k_no_slc_G = np.delete(k_no_slc_G, k_col, axis=1)

In [ ]:
# Obstacle is "buffered" by non-k-sliceable part of FRS
buff_obs_c = obs[:,0][:,None] - c
buff_obs_G = np.hstack((obs[:,1:], k_no_slc_G))
buff_obs_G = remove_zero_columns(buff_obs_G)
buff_obs = np.hstack((buff_obs_c, buff_obs_G))

Test zonotope to halfspace conversion

In [ ]:
buff_obs = Zonotope(buff_obs_c, buff_obs_G)

In [ ]:
A_obs, b_obs = buff_obs.halfspace()

In [ ]:
a = np.array([0,0,1])[:,None]
np.kron(np.eye(3),a)

In [ ]:
Z = buff_obstacle
c = Z[:,0][:,None]
G = Z[:,1:]

(dim, n_gen) = G.shape

In [ ]:
#if dim > 1:

# Build C matrices
#if dim == 2:
C = G
C = np.vstack((-C[1,:], C[0,:]))# get perpendicular vector

In [ ]:
# if dim == 3:
comb = list(itertools.combinations(np.arange(n_gen), dim-1))

In [ ]:
# Normalize normal vectors
C = np.divide(C, np.linalg.norm(C, axis=0)).T

In [ ]:
# Compute dPos, dNeg
d = C @ c

In [ ]:
A = np.vstack((C, -C))
b = np.vstack((d + deltaD, -d + deltaD))

In [ ]:
b

In [ ]:
zero_idx = np.argwhere(np.all(A[...,:]==0, axis=0))

In [ ]:
np.delete(A, zero_idx, axis=1)